## Finite Difference - European Options

- Explicit
- Implicit
- Crank-Nicolson

In [22]:
from typing import Tuple
import numpy as np

K = 100
r = 0.1
q = 0.15
sigma = 0.2
T = 1.0

S0 = 100
M  = 300 # Space grid points
N = 10000 # Time grid points

In [23]:
# Central difference for t 
# Central difference for S

def bs_call_pde_fd_cn(
        S0: float, 
        K: float, 
        r: float, 
        q: float, 
        sigma: float, 
        T: float,
        M: int = 300,         
        N: int = 300            
        ) -> Tuple[float, np.ndarray, np.ndarray, np.ndarray]:
    
    Smax = S0*5
    
    # Space grid -> M --> i
    S = np.linspace(start=0.0, stop=Smax, num=M+1)
    
    # Time grid -> N --> n
    t = np.linspace( start=0.0, stop=T, num=N+1)
    dt = t[1]-t[0]
    
    # Terminal condition i.e. value of option at maturity
    # This is the starting point for the backward evolution.
    V = np.maximum(S - K, 0.0)  

    # Store the full surface for visualization
    V_surface = np.zeros((N+1, M+1))
    V_surface[-1, :] = V.copy()

    # Coefficient for Tridiagonal Matrix
    i = np.arange(start=1, stop=M)
    alpha = 0.25*dt*(sigma**2*(i**2) - (r-q)*i)
    beta  = -0.5*dt*(sigma**2*(i**2) + r)
    gamma = 0.25*dt*(sigma**2*(i**2) + (r-q)*i)

    # AVn=BVn+1
    # Asub​V^n_i−1​+Adiag​V^n_i​+Asup​V^n_i+1​ = Bsub​V^n+1_i−1​+Bdiag​V^n+1_n​+Bsup​V^n+1_i+1​.
    A_sub = -alpha.copy()
    A_diag = 1 - beta.copy()
    A_sup = -gamma.copy()

    B_sub = alpha.copy()
    B_diag = 1 + beta.copy()
    B_sup = gamma.copy()


    from numpy.linalg import solve
    # i --> space S (M)
    # n --> time t
    
    # We move backwards in time, starting from maturity (N) to now (0)
    for n in range(N, 0, -1):
        # At each iteration, we have the grid V^n+1 --> known from the next/later time
        # We compute V_n --> the unknown, earlier time by solving a linear system.

        # Build the right hand side of Crank-Nicholson (EXPLICIT)
        rhs = B_sub*V[i-1] + B_diag*V[i] + B_sup*V[i+1]
        
        # Boundary: V0 and Vmax are the known boundary values at this time level t_n
        t_now = (n-1)*dt
        # V(t_n,0)
        V0 = 0.0
        # V(t_n,S_max) --> linear, Using last value of S
        Vmax = S[-1]*np.exp(-q*(T - t_now)) - K*np.exp(-r*(T - t_now))

        # We "correct" the system to account for known edge values. 
        # Without this, the first and last interior equations would incorrectly include the unknown boundaries.
        rhs[0]  -= A_sub[0]*V0
        rhs[-1] -= A_sup[-1]*Vmax

        # Build IMPLICIT matrix
        # M-1 because solve only for interior nodes i=1...M-1 
        # since boundaries V0 and VM are known and excluded from the system 
        A = np.zeros((M-1, M-1))
        np.fill_diagonal(A, A_diag) # Central point
        np.fill_diagonal(A[1:], A_sub[1:]) # Forward neighbor
        np.fill_diagonal(A[:,1:], A_sup[:-1]) # Backward neighbor

        V_in = solve(A, rhs)

        # After each time n, full profile V_n=[V^0n_​,V^1_n​,…,V^M−1_n​,V^M_n​].
        V[0] = V0
        V[1:M] = V_in
        V[M] = Vmax

        V_surface[n-1, :] = V.copy()

    # Interpolate the option price at the current stock price
    # Value of option at time zero for S
    price = np.interp(S0, S, V)

    # V is just the last profile
    # V_surface contains all profiles
    
    return price, S, t, V, V_surface

In [24]:
# Backward difference approximation for t 
# Central difference for S

def bs_call_pde_fd_explicit(
    S0: float, 
    K: float, 
    r: float, 
    q: float, 
    sigma: float, 
    T: float,
    M: int = 300,
    N: int = 300 
    ) -> Tuple[float, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    
    Smax = S0*5
    
    # Space grid -> M --> i
    S = np.linspace(0.0, Smax, M + 1)
    
    # Time grid -> N --> n
    t = np.linspace(0.0, T, N + 1)
    dt = t[1] - t[0]
    
    # Terminal condition i.e. value of option at maturity
    # This is the starting point for the backward evolution.
    V = np.maximum(S - K, 0.0)
    
    # Store the full surface for visualization
    V_surface = np.zeros((N + 1, M + 1))
    V_surface[-1, :] = V.copy()

    # Coefficient for Tridiagonal Matrix
    i = np.arange(1, M)
    alpha = 0.5 * dt * (sigma ** 2 * (i ** 2) - (r - q) * i)
    beta = 1 - dt * (sigma ** 2 * (i ** 2) + r)
    gamma = 0.5 * dt * (sigma ** 2 * (i ** 2) + (r - q) * i)

    # We move backwards in time, starting from maturity (N) to now (0)
    for n in range(N, 0, -1):

        # Boundary: V0 and Vmax are the known boundary values at this time level t_n
        t_now = (n - 1) * dt
        V0 = 0.0
        Vmax = S[-1] * np.exp(-q * (T - t_now)) - K * np.exp(-r * (T - t_now))
        
        # Backward Discretization for time
        # using information at time n and three values of V
        V_new = V[i] * beta + V[i - 1] * alpha + V[i + 1] * gamma
        
        V[0] = V0
        V[1:M] = V_new
        V[M] = Vmax
        
        V_surface[n - 1, :] = V.copy()
    
    price = np.interp(S0, S, V)
    return price, S, t, V, V_surface

In [25]:
# Forward difference approximation for t 
# Central difference for S

def bs_call_pde_fd_implicit(
    S0: float, 
    K: float, 
    r: float, 
    q: float, 
    sigma: float, 
    T: float,
    M: int = 300, 
    N: int = 300
    ) -> Tuple[float, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:

    Smax = S0*5

    # Space grid -> M --> i
    S = np.linspace(0.0, Smax, M + 1)
    
    # Time grid -> N --> n
    t = np.linspace(0.0, stop=T, num=N + 1)
    dt = t[1] - t[0]
    
    # Terminal condition i.e. value of option at maturity
    # This is the starting point for the backward evolution.
    V = np.maximum(S - K, 0.0)
    
    # Store the full surface for visualization
    V_surface = np.zeros((N + 1, M + 1))
    V_surface[-1, :] = V.copy()
    
    # Coefficient for Tridiagonal Matrix
    i = np.arange(1, M)
    alpha = -0.5 * dt * (sigma ** 2 * (i ** 2) - (r - q) * i)
    beta = 1 + dt * (sigma ** 2 * (i ** 2) + r)
    gamma = -0.5 * dt * (sigma ** 2 * (i ** 2) + (r - q) * i)
    
    from numpy.linalg import solve

    # We move backwards in time, starting from maturity (N) to now (0)
    for n in range(N, 0, -1):
        
        # Boundary: V0 and Vmax are the known boundary values at this time level t_n
        t_now = (n - 1) * dt
        V0 = 0.0
        Vmax = S[-1] * np.exp(-q * (T - t_now)) - K * np.exp(-r * (T - t_now))

        # Build IMPLICIT matrix
        A = np.zeros((M - 1, M - 1))
        np.fill_diagonal(A, beta)
        np.fill_diagonal(A[1:], alpha[1:])
        np.fill_diagonal(A[:, 1:], gamma[:-1])
        
        # Forward Discretization for time:
        # Use one value today
        rhs = V[i]
        rhs[0] -= alpha[0] * V0
        rhs[-1] -= gamma[-1] * Vmax
        
        V_in = solve(A, rhs)
        
        V[0] = V0
        V[1:M] = V_in
        V[M] = Vmax
        
        V_surface[n - 1, :] = V.copy()
    
    price = np.interp(S0, S, V)
    return price, S, t, V, V_surface

In [26]:
# Run all three methods
fd_explicit_price, _, _, _, _ = bs_call_pde_fd_explicit(S0, K, r, q, sigma, T, M, N)
fd_implicit_price, _, _, _, _ = bs_call_pde_fd_implicit(S0, K, r, q, sigma, T,  M, N)
fd_cn_price, _, _, _, _ = bs_call_pde_fd_cn(S0, K, r, q, sigma, T, M, N)

print(f"FD Explicit : {fd_explicit_price:.6f}")
print(f"FD Implicit : {fd_implicit_price:.6f}")
print(f"FD CN       : {fd_cn_price:.6f}")

FD Explicit : 5.037016
FD Implicit : 5.036822
FD CN       : 5.036919
